In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

In [2]:
df_train = dataset["train"].to_pandas()
df_train.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [3]:
from bullet.core.sentiment import SentimentClassifier

import logging

import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

df_train_sample = df_train.sample(n = 10)

classifier = SentimentClassifier()
result = classifier.predict_pandas(df_train_sample.reset_index().rename(columns = {"index": "id"}))

INFO:root:Before: {'id': 12616, 'text': 'Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and porno sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoyable movie.'}
INFO:root:After validation: {'id': 12616, 'text': 'Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and porno sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoyable movie.'}
INFO:root:{'id': 12616, 'text': 'Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and porno sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoyable movie.'}
INFO:root:Prompt output: 
        Please correctly classify each and every movie revie

In [4]:
result

[ClassificationResponse(response='\n[{"id": "12616", "label": "POS"}]', response_dict={'id': '12616', 'label': 'POS'}, id='12616', label='POS'),
 ClassificationResponse(response='\n[{"id": 14288, "label": "POS"}]', response_dict={'id': 14288, 'label': 'POS'}, id=14288, label='POS'),
 ClassificationResponse(response='\n[{"id": 5413, "label": "NEG"}]', response_dict={'id': 5413, 'label': 'NEG'}, id=5413, label='NEG'),
 ClassificationResponse(response='\n[{"id": 9578, "label": "NEG"}]', response_dict={'id': 9578, 'label': 'NEG'}, id=9578, label='NEG'),
 ClassificationResponse(response='\n[{"id": 21023, "label": "POS"}]', response_dict={'id': 21023, 'label': 'POS'}, id=21023, label='POS'),
 ClassificationResponse(response='\n[{"id": 15878, "label": "POS"}]', response_dict={'id': 15878, 'label': 'POS'}, id=15878, label='POS'),
 ClassificationResponse(response='\n[{"id": 5026, "label": "NEG"}]', response_dict={'id': 5026, 'label': 'NEG'}, id=5026, label='NEG'),
 ClassificationResponse(respon

In [5]:
import pandas as pd

dict_pd = []

for item in result:
    payload = {
        "index": item.id,
        "predicted": item.label
    }
    dict_pd.append(payload)

df_predicted = pd.DataFrame.from_records(dict_pd)
df_merge = pd.merge(
    left = df_train_sample.reset_index(),
    right = df_predicted,
    on = "index"
)

df_merge["predicted"] = [0 if predicted == "NEG" else 1 for predicted in df_merge.predicted.values]
df_merge.head()

,index,text,label,predicted
0,14288,I'm normally a sucker for romantic films which...,1,1
1,5413,"It looks to me as if the creators of ""The Clas...",0,0
2,9578,This ludicrous film offers the standard 1970's...,0,0
3,21023,Surprised to see the rather low score for this...,1,1
4,15878,"""The Garden of Allah"" was one of the first fea...",1,1


In [6]:
from sklearn.metrics import classification_report

report = classification_report(df_merge.label, df_merge.predicted)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         4

    accuracy                           1.00         9
   macro avg       1.00      1.00      1.00         9
weighted avg       1.00      1.00      1.00         9



In [7]:
df_train_sample = df_train.sample(n = 1000)

result = await classifier.apredict(
    text = list(df_train_sample.text.values),
    parallelism = 10
)

INFO:root:Input text: ["This was a really cool movie. It just goes to prove that you don't need silly litle things like continuity and scripts to make a movie. It traverses continents in seconds, people get shot and nothing happens to them, swords set on fire, samuari fight on sinking galleons, David Essex is the epitome of slimey villainy and John Rhys Davies is just the dude. I enjoyed this movie but I like s**t movies, this is the perfect example of a very s**t movie that just KICKS ASS. If you like Battlefield Earth you'll love this film, its swashbuckling, its fast, its silly, its samuaraitastic!!!!!!!!!!!!!!!!<br /><br />It also looks as if it was made in 1972"
 'As you probably already know, Jess Franco is one prolific guy. Hes made hundreds upon hundreds of films, many of which are crap. However, he managed to sneak in an occasionally quality work amongst all the assembly line exploitation. "Succubus" isn\'t his best work (thats either "The Diabolical Dr. Z" or "Vampyros Lesbos

ValidationError: 1 validation error for ZeroShotPrompt
examples
  Value error, unexpected character: line 1 column 1 (char 0) [type=value_error, input_value="This was a really cool m... if it was made in 1972", input_type=str]
    For further information visit https://errors.pydantic.dev/2.4/v/value_error

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7fd0be96c1d0>


In [ ]:
import pandas as pd

df_predicted = pd.DataFrame.from_dict([dict(item) for item in result])
df_train_sample["predicted"] = predicted
df_train_sample.head(10)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(df_train_sample.label, df_train_sample.predicted)
print(report)